# Density Estimation

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

## The Claw Function

We consider the following density, derived from a mixture of six Gaussians,

$$
p(x) = \frac{1}{2} \mathcal{N}(0, 1) + \frac{1}{10} \sum_{j=0}^4 \mathcal{N}((j/2)-1,1/100)
$$

In [ ]:
def claw(x):
    sum_pdf = sum([stats.norm(loc=(j/2-1), scale=0.1).pdf(x) for j in range(5)])
    return stats.norm().pdf(x) / 2 + sum_pdf / 10

def sample_claw(n=1000):
    '''Generate samples via ancestral sampling (i.e. follow the generative process).
    '''
    dist = [stats.norm()] + [stats.norm(loc=(j/2-1), scale=0.1) for j in range(5)]
    c_dist = stats.multinomial(1, np.array([1/2, 1/10, 1/10, 1/10, 1/10, 1/10]))
    c_sample = c_dist.rvs(size=n).argmax(axis=-1)
    return np.array([dist[c].rvs() for c in c_sample])

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(-3, 3, 1000)
pdf = claw(x)

ax.plot(x, pdf)

In [ ]:
fig, ax = plt.subplots()

samples = sample_claw()[:, np.newaxis]
ax.scatter(samples, claw(samples), s=5, color='red')

The big picture question that density estimation answers is -- given samples from some underlying generative process, can we recover the process? We are moving from the supervised learning setting to an unsupervised learning setting. Now, we must discover the structure in the data without some form of ground truth labels.

## Histograms

We have infact always been using the simplest density estimator -- the histogram!

In [ ]:
fig, axes = plt.subplots(figsize=(10,10), nrows=2, ncols=2, sharex=True)

bins = [250, 100, 50, 10]
for i in range(2):
    for j in range(2):
        axes[i,j].hist(samples, bins=bins[i*2 + j])

        axes[i,j].set_title(f'Bins = {bins[i*2 + j]}')
        axes[i,j].set_ylabel('Count')
        if i > 0:
            axes[i,j].set_xlabel(r'$x$')
fig.tight_layout()

Clearly, having finer bins can reveal finer structures with sufficient number of samples. This can, however, quickly turn bad because choosing bins in higher dimensional spaces is equivalent to picking hypercubes, which is tricky.

## Kernel Density Estimators

These are non-parametric density estimators, often useful mostly in low dimensions. We place _smoothing kernels_ $K$ centered at each sample, where the coverage is specified by a bandwidth parameter $h$. In general, the estimated density function is then given by

$$
\widehat{p}(x) = \frac{1}{n} \sum_{i=1}^n K_h(\lvert x - x_i \rvert)
$$

A common choice of a _smoothing kernels_ (i.e $\int K(z) dz = 1$) is the Gaussian kernel,

$$
K_h(z) = \frac{1}{\sqrt{2\pi h}}\exp{\left\{-\frac{z^2}{2h} \right\} }
$$

The bandwidth parameter helps us balance the bias-variance trade-off.

**NOTE**: You may encounter the term kernel used in different contexts across machine learning, e.g. a _positive-definite kernel_, _Mercer kernel_, etc. These are different from _smoothing kernels_ used here.


In [ ]:
from sklearn.neighbors import KernelDensity

fig, axes = plt.subplots(figsize=(10,10), nrows=2, ncols=2, sharex=True, sharey=True)

bw = [0.005, 0.05 , 0.1, 1.]
for i in range(2):
    for j in range(2):
        kde = KernelDensity(bandwidth=bw[i * 2 + j])
        kde.fit(samples)

        axes[i,j].plot(x[:,np.newaxis], np.exp(kde.score_samples(x[:,np.newaxis])))

        axes[i,j].set_title(fr'$h = {bw[i * 2 + j]}$')
        if i > 0:
            axes[i,j].set_xlabel(fr'$x$')
        if j < 1:
            axes[i,j].set_ylabel(fr'$p(x)$')
fig.tight_layout()


Evidently, we see that on one hand smaller bandwidths create overly complicated distributions, but larger bandwidths create overly simplistic distributions.

## Gaussian Mixture Models

Gaussian Mixture Models (GMMs) are a class of discrete _latent_ variable models, described by the following marginal density over the input data

$$
p(x) = \sum_{k=1}^K p(z = k) p(x\mid z = k)
$$

where $p(x\mid z = k) = \mathcal{N}(x; \mu_k, \Sigma_k)$ is the $k^{\mathrm{th}}$ Gaussian component in the mixture. Clearly, $0 < p(z=k) < 1$, and $\sum_{k=1}^K p(z = k) = 1$. In theory, GMMs can approximate any smooth density, i.e. they are universal approximators of density functions. In practice, however, this property is harder to utilize due to light tails and being "too smooth".

For a given set of samples $\left\{x_i\right\}_{i=1}^N$, we aim to maximize the $\log$-likelihood using the density function $p(x)$. This involves finding each mixing paramter $p(z = k)$, and the mean and covariances for each $p(x \mid z=k)$.

### Old Faithful Dataset

This dataset has been pre-processed to be zero mean and unit variance.

In [ ]:
import pandas as pd
X = pd.read_csv('oldfaithful.csv').values

fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(X[:, 0], X[:, 1])

We solve a constrained optimization problem via the method of Lagrange multipliers. It is, however, not feasible to maximize this in closed-form. Nevertheless, we can build an iterative procedure after some algebraic manipulations, which boils down to the alternating between the following steps:

**E step**:

$$
\gamma(z_{ik}) \propto p(z=k)p(x_i \mid z = k)
$$

**M step**:

$$
\begin{aligned}
n_k &= \sum_{i=1}^n \gamma(z_{ik}) \\
\mu_{k}^{\mathrm{new}} &= \frac{1}{n_k} \sum_{i=1}^n \gamma(z_{ik}) x_i \\
\Sigma_{k}^{\mathrm{new}} &= \frac{1}{n_k} \sum_{i=1}^n \gamma(z_{ik}) (x_i - \mu_{k}^{\mathrm{new}})(x_i - \mu_{k}^{\mathrm{new}})^T \\
p^{\mathrm{new}}(z = k) &= \frac{n_k}{n}
\end{aligned}
$$

Intuitively, these update steps first compute the contribution of each cluster component towards explaining a datapoint. Finally, using these contribution weights, we compute the new mixing coefficients, and the means and covariances.

This iterative procedure is known as **Expectation-Maximization** (EM). The procedure is more generally applicable than just for GMMs -- typically for problems where maximizing the _complete_ distribution (the full joint) is easier than maximizing the _marginal_ (the algorithm was introduced for a missing data setting first).

In [ ]:
from scipy import stats


def gmm_lik(X, pi, mu, cov):
    '''
    X: N x D
    pi: K
    mu: K x D
    cov: K x D x D
    '''
    dists = [stats.multivariate_normal(mean=mu[k], cov=cov[k]) for k in range(K)]
    pdf = np.c_[[dist.pdf(X) for dist in dists]].T  ## N x K
    lik = np.sum(pdf * pi[np.newaxis, :], axis=-1)
    return lik

def expectation(X, pi, mu, cov):
    '''
    X: N x D
    pi: K
    mu: K x D
    cov: K x D x D
    '''
    K = len(pi)
    dists = [stats.multivariate_normal(mean=mu[k], cov=cov[k]) for k in range(K)]
    pdf = np.c_[[dist.pdf(X) for dist in dists]].T  ## N x K

    gamma = pdf * pi[np.newaxis, ...]  ## N x K
    gamma = gamma / np.sum(gamma, axis=-1, keepdims=True)

    return gamma

def maximization(X, gamma):
    '''
    X: N x D
    gamma: N x K
    '''
    N, K = gamma.shape
    N_k = np.sum(gamma, axis=0) ## K
    
    mu_new = np.c_[[np.sum(gamma[:, k, np.newaxis] * X, axis=0) / N_k[k] for k in range(K)]]  ## K x D

    X_c = [X - mu_new[k][np.newaxis, :] for k in range(K)]
    cov_new = np.c_[[np.matmul(X_c[k].T, gamma[:, k, np.newaxis] * X_c[k]) / N_k[k] for k in range(K)]]  ## K x D x D

    pi = N_k / N

    return pi, mu_new, cov_new

### Visualizing Training of GMM

In [ ]:
from sklearn.cluster import KMeans
from scipy.special import logsumexp

K = 2
D = 2
pi = np.ones(K) / K

# mu = 2. * np.random.randn(K, D)
mu = np.array([[1., -1.], [-1., 1.5]])

# kmeans = KMeans(n_clusters=K).fit(X)
# mu = kmeans.cluster_centers_

cov = np.c_[[np.eye(D,D) for k in range(K)]]

ckpts_i = [1, 5, 10, 15, 20]
ckpts = []

for i in range(20):
    ## store desired checkpoints for plotting.
    if i + 1 in ckpts_i:
        ckpts.append((np.copy(pi), np.copy(mu), np.copy(cov)))

    gamma = expectation(X, pi, mu, cov)
    pi, mu, cov = maximization(X, gamma)

In [ ]:
fig, axes = plt.subplots(figsize=(5,5 * len(ckpts_i)), nrows=len(ckpts_i), sharex=True, sharey=True)

xx, yy = np.meshgrid(np.linspace(-2.1, 2.1, 200), np.linspace(-2.1, 2.1, 200))
mesh_X = np.concatenate([xx[..., np.newaxis], yy[..., np.newaxis]], axis=-1).reshape(-1, 2)

for ax, i, (pi, mu, cov) in zip(axes, ckpts_i, ckpts):
    ax.contourf(xx, yy, gmm_lik(mesh_X, pi, mu, cov).reshape(*xx.shape), cmap=plt.cm.coolwarm)
    X_lik = gmm_lik(X, pi, mu, cov)
    ax.scatter(X[:, 0], X[:, 1], c=X_lik, s=7, cmap=plt.cm.viridis)
    ax.set_title(f'Iteration {i}')
    ax.set_ylabel(fr'$p(x)$')
    ax.set_aspect('equal')
fig.tight_layout()

## Fitting GMMs with scikit-learn

`scikit-learn` implements the above algorithm as well. In particular, it allows us to use k-Means cluster initialization for potentially faster convergence. It also allows for other kinds of parametrizations, e.g. a "tied" parametrization which shares a "full" covariance matrix across all components of the mixture.

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
fig, axes = plt.subplots(figsize=(5,20), nrows=4, sharex=True, sharey=True)

for ax, ctype in zip(axes, ['full', 'tied', 'diag', 'spherical']):
    gm = GaussianMixture(n_components=K, covariance_type=ctype).fit(X)

    ax.contourf(xx, yy, np.exp(gm.score_samples(mesh_X)).reshape(*xx.shape), cmap=plt.cm.coolwarm)
    ax.scatter(X[:, 0], X[:, 1], c=np.exp(gm.score_samples(X)), s=7, cmap=plt.cm.viridis)
    ax.set_title(f'Covariance type = {ctype}')
    ax.set_ylabel(fr'$p(x)$')
    ax.set_aspect('equal')

fig.tight_layout()